# Utility functions and configs

Run below cell befor start capturing packets.

In [1]:
from scapy.all import *
import pandas as pd # Pandas - Create and Manipulate DataFrames
import binascii # Binary to Ascii 
%matplotlib inline

def pkt_callback(pkt):
    pkt.show() # debug statement

# Collect field names from IP/TCP/UDP (These will be columns in DF)
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]
icmp_fields = [field.name for field in ICMP().fields_desc]


dataframe_fields = ip_fields + ['time'] + tcp_fields + icmp_fields + ['payload','payload_raw','payload_hex']

def pretty(pcap):
    # Create blank DataFrame
    df = pd.DataFrame(columns=dataframe_fields)
    
    for packet in pcap[IP]:
        # Field array for each row of DataFrame
        field_values = []
        # Add all IP fields to dataframe
        for field in ip_fields:
            if field == 'options':
                # Retrieving number of options defined in IP Header
                field_values.append(len(packet[IP].fields[field]))
            elif field == 'proto':
                if(packet[IP].fields[field]== 17):
                    field_values.append('udp')
                elif(packet[IP].fields[field]== 6):
                    field_values.append('tcp')
                elif(packet[IP].fields[field]==1):
                    field_values.append('icmp')
                else :
                    field_values.append(packet[IP].fields[field])
            else:
                field_values.append(packet[IP].fields[field])
    
        field_values.append(packet.time)
    
        layer_type = type(packet[IP].payload)
        for field in tcp_fields:
            try:
                if field == 'options':
#                    field_values.append(len(packet[layer_type].fields[field]))
                    field_values.append(packet[layer_type].fields[field][0])
                elif field == 'seq':
                    seq_num = str(int(packet[layer_type].fields[field])%10000)
                    field_values.append(seq_num)
                elif field == 'ack':
                    ack_num = str(int(packet[layer_type].fields[field])%10000)
                    field_values.append(ack_num)
                else:
                    field_values.append(packet[layer_type].fields[field])
            except Exception, e:
                print ("exp message: "+ str(e))
                field_values.append(None)
            #except:
            #    field_values.append(None)

        
        for field in icmp_fields:
            try:
                if field == 'options':
                    field_values.append(packet[layer_type].fields[field][0])
                elif field == 'code':
                    if packet[layer_type].fields[field] == 3 :
                        field_values.append('port-unreachable')
                    else :
                        field_values.append(packet[layer_type].fields[field])
                elif field == 'type':
                    if packet[layer_type].fields[field] == 3 :
                        field_values.append('dest-unreach')
                    else :
                        field_values.append(packet[layer_type].fields[field])
                else:
                    field_values.append(packet[layer_type].fields[field])
            except:
                field_values.append(None)
    
        # Append payload
        field_values.append(len(packet[layer_type].payload))
        field_values.append(packet[layer_type].payload.original)
        field_values.append(binascii.hexlify(packet[layer_type].payload.original))
        # Add row to DF
        df_append = pd.DataFrame([field_values], columns=dataframe_fields)
        df = pd.concat([df, df_append], axis=0)
    # Reset Index
    df = df.reset_index()
    # Drop old index column
    df = df.drop(columns="index")
    return df

### Exercise One ,TCP Handshake packets

In [2]:
pcap = sniff(iface="s1-eth1", filter="tcp",count=3)

df = pretty(pcap)

# Return Src Address, Dst Address, Src Port, Dst Port, IP Flags, TCP Flags and MSS
df[['src','dst','sport','dport','flags','seq','ack','options']]

,src,dst,sport,dport,flags,flags,seq,seq,ack,options,options
0,10.0.0.1,10.0.0.2,39432,5005,DF,S,6185,2074546185,0,0,"(MSS, 1460)"
1,10.0.0.2,10.0.0.1,5005,39432,DF,SA,3044,1175743044,6186,0,"(MSS, 1460)"
2,10.0.0.1,10.0.0.2,39432,5005,DF,A,6186,2074546186,3045,0,"(NOP, None)"


### Exercise Two ,UDP

In [37]:
pcap = sniff(iface="s1-eth1", filter="udp or icmp",count=2)

df = pretty(pcap)

# Return Src Address, Dst Address, Src Port, Dst Port, IP Flags, TCP Flags and MSS
df[['src','dst','proto','sport','dport','type','code']]

,src,dst,proto,sport,dport,type,code
0,10.0.0.1,10.0.0.2,udp,56636,8000,None,None
1,10.0.0.2,10.0.0.1,icmp,None,None,dest-unreach,port-unreachable


### Exercise Two ,TCP

In [25]:
pcap = sniff(iface="s1-eth1", filter="tcp or icmp",count=2)

df = pretty(pcap)

# remove ICMP seq number and IP flags fields
df_dup = df.loc[:,df.columns.duplicated()]
df = df.loc[:,~df.columns.duplicated()]
df['flags'] = df_dup['flags']

# Return Src Address, Dst Address, Src Port, Dst Port, IP Flags, TCP Flags and MSS
df[['src','dst','proto','sport','dport','flags']]

,src,dst,proto,sport,dport,flags,flags
0,10.0.0.1,10.0.0.2,tcp,57338,5005,DF,S
1,10.0.0.2,10.0.0.1,tcp,5005,57338,DF,RA


### Exercise Four ,Sample One

In [76]:
pcap = sniff(iface="s1-eth1", filter="tcp",count=38)

df = pretty(pcap)

# remove ICMP seq number and IP flags fields
df_dup = df.loc[:,df.columns.duplicated()]
df = df.loc[:,~df.columns.duplicated()]
df['flags'] = df_dup['flags']

# Return Src Address, Dst Address, Src Port, Dst Port, IP Flags, TCP Flags and MSS
df[['src','dst','proto','sport','dport','flags','seq','ack','options']]

,src,dst,proto,sport,dport,flags,seq,ack
0,10.0.0.1,10.0.0.2,tcp,39444,5005,S,6231,0
1,10.0.0.2,10.0.0.1,tcp,5005,39444,SA,9318,6232
2,10.0.0.1,10.0.0.2,tcp,39444,5005,A,6232,9319
3,10.0.0.1,10.0.0.2,tcp,39444,5005,PA,6232,9319
4,10.0.0.2,10.0.0.1,tcp,5005,39444,A,9319,6245
5,10.0.0.1,10.0.0.2,tcp,39444,5005,PA,6245,9319
6,10.0.0.2,10.0.0.1,tcp,5005,39444,A,9319,6258
7,10.0.0.1,10.0.0.2,tcp,39444,5005,PA,6258,9319
8,10.0.0.2,10.0.0.1,tcp,5005,39444,A,9319,6271
9,10.0.0.1,10.0.0.2,tcp,39444,5005,PA,6271,9319


### Exercise Four ,Sample Two

In [77]:
pcap = sniff(iface="s1-eth1", filter="tcp",count=38)

df = pretty(pcap)

# remove ICMP seq number and IP flags fields
df_dup = df.loc[:,df.columns.duplicated()]
df = df.loc[:,~df.columns.duplicated()]
df['flags'] = df_dup['flags']

# Return Src Address, Dst Address, Src Port, Dst Port, IP Flags, TCP Flags and MSS
df[['src','dst','proto','sport','dport','flags','seq','ack','options']]

,src,dst,proto,sport,dport,flags,seq,ack
0,10.0.0.1,10.0.0.2,tcp,39878,5005,S,5498,0
1,10.0.0.2,10.0.0.1,tcp,5005,39878,SA,9557,5499
2,10.0.0.1,10.0.0.2,tcp,39878,5005,A,5499,9558
3,10.0.0.1,10.0.0.2,tcp,39878,5005,PA,5499,9558
4,10.0.0.1,10.0.0.2,tcp,39878,5005,FPA,5512,9558
5,10.0.0.2,10.0.0.1,tcp,5005,39878,A,9558,5512
6,10.0.0.2,10.0.0.1,tcp,5005,39878,FA,9558,5708
7,10.0.0.1,10.0.0.2,tcp,39878,5005,A,5708,9559
8,10.0.0.1,10.0.0.2,tcp,39888,5005,S,2824,0
9,10.0.0.2,10.0.0.1,tcp,5005,39888,SA,8119,2825


### Exercise Four ,Sample Three

In [78]:
pcap = sniff(iface="s1-eth1", filter="tcp",count=38)

df = pretty(pcap)

# remove ICMP seq number and IP flags fields
df_dup = df.loc[:,df.columns.duplicated()]
df = df.loc[:,~df.columns.duplicated()]
df['flags'] = df_dup['flags']

# Return Src Address, Dst Address, Src Port, Dst Port, IP Flags, TCP Flags and MSS
df[['src','dst','proto','sport','dport','flags','seq','ack','options']]

,src,dst,proto,sport,dport,flags,seq,ack
0,10.0.0.1,10.0.0.2,tcp,39926,5005,S,3963,0
1,10.0.0.2,10.0.0.1,tcp,5005,39926,SA,7483,3964
2,10.0.0.1,10.0.0.2,tcp,39926,5005,A,3964,7484
3,10.0.0.1,10.0.0.2,tcp,39926,5005,PA,3964,7484
4,10.0.0.2,10.0.0.1,tcp,5005,39926,A,7484,3977
5,10.0.0.1,10.0.0.2,tcp,39926,5005,PA,3977,7484
6,10.0.0.2,10.0.0.1,tcp,5005,39926,A,7484,3990
7,10.0.0.1,10.0.0.2,tcp,39926,5005,PA,3990,7484
8,10.0.0.2,10.0.0.1,tcp,5005,39926,A,7484,4003
9,10.0.0.1,10.0.0.2,tcp,39926,5005,PA,4003,7484
